# Colab × GitHub (шаблон с SSH-ключами)

Цель: работа с GitHub через SSH. Ключи храним в Google Drive, чтобы сохранялись между сессиями.

Алгоритм:
1. Смонтировать Drive.
2. Сгенерировать ключ (или использовать существующий) в `MyDrive/.ssh`.
3. Добавить публичный ключ в GitHub → Settings → SSH and GPG keys.
4. Клонировать репозиторий по SSH и работать как обычно.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [56]:
import subprocess, pathlib

def sh(cmd, cwd=None, check=True):
    return subprocess.run(cmd, cwd=cwd, shell=True, text=True, capture_output=True, check=check)

def sh_live(cmd, cwd=None):
    """Выполнить команду и сразу печатать stdout/stderr в ячейку"""
    return subprocess.run(cmd, cwd=cwd, shell=True)

def git_config(user_name: str, user_email: str):
    sh(f'git config --global user.name "{user_name}"')
    sh(f'git config --global user.email "{user_email}"')
    sh('git config --global credential.helper ""')

git_config("SIA86", "sfron4ik@gmail.com")

In [57]:
from pathlib import Path

DRIVE_SSH_DIR = Path('/content/drive/MyDrive/.ssh')
DRIVE_SSH_DIR.mkdir(parents=True, exist_ok=True)
ID_KEY = DRIVE_SSH_DIR / 'id_ed25519'
ID_PUB = DRIVE_SSH_DIR / 'id_ed25519.pub'

if not ID_KEY.exists():
    sh(f'ssh-keygen -t ed25519 -N "" -f "{ID_KEY}"')
    print("Сгенерирован новый ключ.")
else:
    print("Найден существующий ключ.")

sh('mkdir -p ~/.ssh && chmod 700 ~/.ssh')
sh(f'ln -sf "{ID_KEY}" ~/.ssh/id_ed25519')
sh(f'ln -sf "{ID_PUB}" ~/.ssh/id_ed25519.pub')
sh('chmod 600 ~/.ssh/id_ed25519 && chmod 644 ~/.ssh/id_ed25519.pub')
sh('ssh-keyscan github.com >> ~/.ssh/known_hosts || true')
sh('chmod 644 ~/.ssh/known_hosts')

print("Публичный ключ для GitHub:")
print(ID_PUB.read_text())

Найден существующий ключ.
Публичный ключ для GitHub:
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIKot/XiGZ9/rvjmmZsCuGn1O0CIQzGUeiYLjyWCpdWN8 root@01bff0625f36



In [58]:
res = sh('ssh -o StrictHostKeyChecking=yes -T git@github.com', check=False)
print(res.stdout or res.stderr)

Hi SIA86! You've successfully authenticated, but GitHub does not provide shell access.



In [ ]:
OWNER_AND_REPO = "SIA86/pretrained_rl_ppo_model"  # например: octocat/Hello-World
DEST_DIR = "/content/drive/MyDrive/stocks/pretrained_rl_ppo"
ssh_url = f"git@github.com:{OWNER_AND_REPO}.git"
sh(f'git clone "{ssh_url}" "{DEST_DIR}"')

CompletedProcess(args='git clone "git@github.com:SIA86/pretrained_rl_ppo_model.git" "/content/drive/MyDrive/stocks/pretrained_rl_ppo"', returncode=0, stdout='', stderr="Cloning into '/content/drive/MyDrive/stocks/pretrained_rl_ppo'...\n")

In [53]:
%cd "$DEST_DIR"
!git status

/content/drive/MyDrive/stocks/pretrained_rl_ppo
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   colab_github_ssh.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	scr/model.py

no changes added to commit (use "git add" and/or "git commit -a")


In [49]:
# Посмотреть ветки
%cd "$DEST_DIR"
!git branch -a

/content/drive/MyDrive/stocks/pretrained_rl_ppo
* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/main


In [50]:
# Подтянуть изменения
%cd "$DEST_DIR"
!git pull

/content/drive/MyDrive/stocks/pretrained_rl_ppo
Already up to date.


In [ ]:
# Переключиться или создать ветку
%cd "$DEST_DIR"
!git checkout -b main

In [55]:
# Добавить все изменения
%cd "$DEST_DIR"
!git add --all

/content/drive/MyDrive/stocks/pretrained_rl_ppo
error: read error while indexing colab_github_ssh.ipynb: Operation not permitted
error: colab_github_ssh.ipynb: failed to insert into database
error: unable to index file 'colab_github_ssh.ipynb'
fatal: updating files failed


In [ ]:
# Сделать коммит
%cd "$DEST_DIR"
!git commit -m "first commit from Colab"

In [ ]:
# Отправить на GitHub
%cd "$DEST_DIR"
!git push -u origin main